Congfigure HF to use right disks on AML

In [11]:
! sudo mkdir /mnt/cache/huggingface
! sudo chown azureuser: /mnt/cache/huggingface
%env HF_HOME=/mnt/cache/huggingface


mkdir: cannot create directory ‘/mnt/cache/huggingface’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


env: HF_HOME=/mnt/cache/huggingface


Login to HF hub to use gated models

In [12]:
from huggingface_hub import login

login()

# Chat inferencing using pipeline

In [13]:
from transformers import pipeline

# To create a pipeline, we need to specify 
# * The **task** we want the pipeline to perform
# * The **model** that should be used
# * The **device** to use (GPU, CPU, NPU etc)
llama2 = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf", device="cuda")

# Llama 2 uses a special specific kind of prompt template. SYS encapsulates the system prompt, INST the system + user prompts (instructions to the model)
prompt = '''
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

What is four plus two? [/INST]
'''

# Now we can call the pipeline with the prompt
print(llama2(prompt))

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

[{'generated_text': "\n<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is four plus two? [/INST]\nHello! I'm here to help you with your question. Four plus two is equal to six. Is there anything else I can help you with? Please feel free to ask!"}]


# Text generation via model.generate

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np

# First, load the model. AutoModelForCausalLM loads the model and attaches a causal language modelling head to the model i.e. makes the model predict the next token.
# We load Mistral-7B (non-chat!)
# Use the first GPU.
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="cuda"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

input = ["The fiddler fanned the flames"]
# Tokenize the input, return tensors in pytorch format
encodeds = tokenizer(input, return_tensors="pt")
# Send the tokenized input to the GPU
model_inputs = encodeds.to("cuda")

# Generate output using the model from the tokenized input. No sampling is used, so the most probable token is picked
generated_ids = model.generate(**model_inputs, max_new_tokens=100)
# Use the tokenizer to decode the model output
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print("No sampling: " + decoded[0])

# Generate output, but this time with sampling and a somewhat high temperature for more interesting output.
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, temperature=1.0)
# Use the tokenizer to decode the model output
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print("With sampling: " + decoded[0])


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


No sampling: The fiddler fanned the flames of the fire, and the flames fanned the fiddler’s soul.

The fiddler fanned the flames of the fire, and the flames fanned the fiddler’s soul.

The fiddler fanned the flames of the fire, and the flames fanned the fiddler’s soul.

The fiddler fanned the flames of the fire, and the flames fanned the fiddler’s soul.


With sampling: The fiddler fanned the flames of the fire with his back.

“We’re safe for now,” he said, glancing up briefly from his work.

The two of us settled around the fire with our provisions of tea, bread, and a little bit of honey.

“Do you believe all the stories?” I asked around a mouthful of bread.

The fiddler shook his head, and I felt a familiar pang of guilt.

“My mother said if you


In [15]:
from transformers import pipeline

# Create a pipeline using the flan-t5 model, use the first GPU
t5 = pipeline(model="google/flan-t5-base", device=0)
# Ask the model to translare a sentence from english to german. Need to use special task prefix in input!
print(t5("translate English to German: How old are you?"))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

[{'generated_text': 'Wie old sind Sie?'}]


/anaconda/envs/hf/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
# Create a pipeline with a toxic / hate speech classifier, for the text-classification task
hatebert = pipeline("text-classification", model="tomh/toxigen_roberta")
# Print the classes for sample sentences
print(hatebert("I love you!"))
print(hatebert("I hate you!"))

config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/anaconda/envs/hf/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.9992910623550415}]
[{'label': 'LABEL_1', 'score': 0.9773984551429749}]
